<center><h1> Aggregating work.

In [1]:
import pandas as pd
import pickle
import random
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',100)
%matplotlib inline
import latex_options
import spacy
import numpy as np

ModuleNotFoundError: No module named 'latex_options'

# Loading datasets.

In [ ]:
manual_data = pd.read_csv("../Data/Manual_labeling.csv")
projectsLabeled = pd.read_csv('../Data/Labeled_Data/projects.csv',encoding='ISO-8859-1')
countries = pd.read_csv('../Data/Labeled_Data/countries.csv', encoding='ISO-8859-1')
newsData = pd.read_pickle('../Data/Feedly_Processed_DF_cleaned.pkl')

In [40]:
projectInfo = pd.read_csv('../Data/EWS_Published Project_Listing_DD.csv',encoding='ISO-8859-1')

# Cleaning datasets.

In [3]:
# Remove the new line arguments from the text.
newsData.article_text = newsData.article_text.apply(lambda x: x.replace('\n', ''))

# Vectorize text data.

In [4]:
nlp = spacy.load('en_core_web_lg')

In [5]:
newsData['title_nlp'] = newsData.title.apply(lambda x: nlp(x))

In [6]:
newsData['text_nlp'] = newsData.article_text.apply(lambda x: nlp(x))

In [14]:
newsData['text_vector'] = newsData.text_nlp.apply(lambda x: x.vector)

# Finding entities.

In [8]:
newsData['text_entities'] = newsData.text_nlp.apply(lambda x: x.ents)

In [47]:
newsData.memory_usage

<bound method DataFrame.memory_usage of       article_id                                              title  \
10900   eebb9702  India, World Bank sign financing agreement for...   
4268    6832ce57  Rs 40000-crore development projects in limbo i...   
1663    30f8f65e  https://www.the-american-interest.com/2018/01/...   
3789    5ec16472  $300 Million to Expand and Upgrade Infrastruct...   
6657    989c9942  ADB Provides $346 Million to Upgrade State Hig...   
4243     67c8512  Roundup: Bangladesh to benefit from China's op...   
1326    2a5eb487  Haiti - Economy : Fruitful meeting between Moï...   
7033    a034930a  IREDA inks 150 million-euro loan agreement wit...   
3240    5353b43c  African Development Bank Grants $324M for Gree...   
2191    3bdc3c9e  Venezuela’s Mining Arc boom sweeps up Indigeno...   
4574    6e9647c1     Paraguay tenders waterworks for eastern cities   
11001   f0e125c1        EBRD provides €6 million loan to PeopleCert   
10264   e1bb22d5  Waste management ru

In [13]:
entities = newsData.iloc[0].text_entities

In [18]:
for ent in entities:
    print(ent.label_, ent.text)

GPE New Delhi
GPE India
DATE Dec 20
ORG IDA Credit
MONEY USD 125 million
WORK_OF_ART Skills Strengthening for Industrial Value Enhancement Operation
ORG the Government of India
ORG the World Bank
DATE Tuesday
PERSON Sameer Kumar Khare
ORG Joint
ORG Department of Economic Affairs
ORG Ministry of Finance
ORG the Government of India
PERSON Hisham Abdo
ORG World Bank (India
ORG the World Bank
DATE November 30, 2022


In [19]:
labels = [ent.label_ for ent in entities]

In [26]:
train_df = projectsLabeled.merge(newsData, on='article_id')

In [31]:
projectsLabeled.columns

Index(['article_id', 'published', 'title', 'url', 'feed_label',
       'ProjectNumber', 'EWS Project Name', 'EWS hyperlink', 'Matched'],
      dtype='object')

In [27]:
train_df.head()

,article_id,published_x,title_x,url_x,feed_label_x,ProjectNumber,EWS Project Name,EWS hyperlink,Matched,title_y,url_y,feed_label_y,content,published_y,summary,article_text,article_keywords,article_text_len,top_lang,title_nlp,text_nlp,text_vector,text_entities
0,10f9ed2,1/11/18,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,"ADB-41123-015, ADB-48158-001, ADB-41435-053",Road Network Improvement Project (formerly Sec...,https://ewsdata.rightsindevelopment.org/projec...,1,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,NaN,2018-01-11 06:27:29,"<table border=""0"" cellspacing=""3"" cellpadding=...",The Asian Development Bank (ADB) today signed ...,"[development, supply, tonle, million, infrastr...",2426,en,"(ADB, Provides, Support, for, Three, Infrastru...","(The, Asian, Development, Bank, (, ADB, ), tod...","[-0.10169418, 0.15952449, 0.004517287, -0.0958...","((The, Asian, Development, Bank), (ADB), (toda..."
1,c0eece9b,5/13/18,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,ADB-47282-001,Energy Supply Improvement Investment Program (...,https://ewsdata.rightsindevelopment.org/projec...,1,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,NaN,2018-05-13 20:00:00,ADB and Afghanistan inaugurated on 7 May a new...,"KABUL, AFGHANISTAN (14 May 2018) — The Asian D...","[helps, network, development, electricity, sou...",2539,en,"(ADB, Helps, Inaugurate, New, Power, Distribut...","(KABUL, ,, AFGHANISTAN, (, 14, May, 2018, ), —...","[-0.094528675, 0.2013577, 0.061322328, -0.0503...","((KABUL), (AFGHANISTAN), (14, May, 2018), (The..."
2,d1d79dd8,2/20/18,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,ADB-50312-003,Railway Rolling Stock Operations Improvement P...,https://ewsdata.rightsindevelopment.org/projec...,1,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,NaN,2018-02-20 19:00:00,ADB's Board of Directors has approved loans to...,"MANILA, PHILIPPINES (21 February 2018) — The B...","[boost, freight, railway, rolling, transport, ...",3867,en,"(ADB, Provides, $, 360, Million, for, Rolling,...","(MANILA, ,, PHILIPPINES, (, 21, February, 2018...","[-0.03437182, 0.14863983, -0.020184146, -0.083...","((MANILA), (PHILIPPINES), (21), (February, 201..."
3,f0d65e5,2/25/18,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,ADB-50410-001,ASEAN Distributed Power Project: Initial Pover...,https://ewsdata.rightsindevelopment.org/projec...,1,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,NaN,2018-02-25 23:57:32,"<table border=""0"" cellspacing=""3"" cellpadding=...","//es_subbox( $namefield = "" "", $desc = ""Subscr...","[bgrimm, support, expansion, southeast, renewa...",289,en,"(ADB, provides, financing, to, Thailand, 's, B...","(//es_subbox, (, $, namefield, =, "", "", ,, $, ...","[-0.14581825, 0.21669994, -0.0136585785, -0.02...","((//es_subbox), (PremiumThe, Asian, Developmen..."
4,4a557358,2/26/18,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,ADB-50410-001,ASEAN Distributed Power Project: Initial Pover...,https://ewsdata.rightsindevelopment.org/projec...,1,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,NaN,2018-02-26 05:12:06,"<table border=""0"" cellspacing=""3"" cellpadding=...",The Asian Development Bank (ADB) has agreed to...,"[asean, region, adbs, bgrimm, support, expansi...",1431,en,"(ADB, 's, $, 235, m, loa

In [ ]:
train_df['title_nlp'] = train_df.title.apply(lambda x: nlp(x))
train_df['text_nlp'] = train_df.article_text.apply(lambda x: nlp(x))
train_df['text_vector'] = train_df.text_nlp.apply(lambda x: x.vector)
train_df['text_entities'] = train_df.text_nlp.apply(lambda x: x.ents)

In [39]:
projectsLabeled.columns

Index(['article_id', 'published', 'title', 'url', 'feed_label',
       'ProjectNumber', 'EWS Project Name', 'EWS hyperlink', 'Matched'],
      dtype='object')

# Merging datasets.